In [282]:
import math
import matplotlib.pyplot as plt
import numpy as np
from typing import Optional

In [283]:
class Car:
    '''
    Car object
    '''
    pass

In [284]:
def __init__(self, id:int, position: float, sec_rule: Optional[np.ndarray] = None, velocity1: float = 60.0, velocity2: float = 20.0, delta_t:float = 1/3600, acc1: float = 0.0, d_min: float = 0.001, max_speed: float = 105.0,front_car = None, a_min: float = -2.0, a_max: float = 3.0, a_emer: float = 5.0):
        '''
        Constructor for car class.

        Args:
        id: int representing the identity number for each car
        velocity: float representing the car's velocity.
        position: float representing the current position of the car
        delta_t: float (default 1/3600) seconds representing the time step.
        d_min: float (default 0.001 km) representing the minimal distance required between front car.
        max_speed: float km/h representing the speed limit (default 105 km/hour) on the road.
        front_car: Car object (default None) representing the car in front.
        aggressive_driver: boolean default False) representing if it is true that the car's driver is aggressive driver.
        #######a_min: float (default -2.0 km/h/s)representing the maximum confortable deceleration.
        #######a_max: float (default 3.0 km/h/s) representing the maximum comfortable acceleration.
        #######a_emer: float (default 5.0 km/h/s) representing maximum emergency deceleration that a car is able to drive on safety.
        '''
        self.id = id
        self.position = position
        self.sec_rule = sec_rule
        self.velocity1 = velocity1
        self.velocity2 = velocity2
        self.delta_t = delta_t
        self.acc1 = acc1
        self.d_min = d_min
        self.max_speed = max_speed
        self.front_car = front_car
        self.a_min = a_min
        self.a_max = a_max
        self.a_emer = a_emer

Car.__init__ = __init__

CALCULATING SECOND RULE OF COMFORT USING CUBIC FUNCTION
- All kinds of safety distances required by the downhill section are the largest, followed by the corresponding flat section, and the smallest on the uphill section.
- The cubic function ensures smooth transitions between different road slopes. It allows customization for aggressive vs. non-aggressive drivers.
- sc =c⋅(road_grade)^3 + t, where c is cubic constant, t is baseline comfort time driving on flat road. 

In [285]:
def compute_cubic_constant_final(self,x:float , y: float, t:float = 3.0):
    '''
    calculate the cubic constant for the following formula:
        c⋅x^3 + t = y
        
    Args:
        x: float representing the maximum road_grade.
        y: float representing expected second rule of space.
        t: float = 3.0 second rule of space when driving on flat ground.

    Return:
        c: float representing the computed constant.
    '''
    return (y - t) / (x ** 3)

Car.compute_cubic_constant_final = compute_cubic_constant_final

In [286]:
def compute_second_rule_comfort_cubic_final(self,road_grade:float, y_uphill: float,y_downhill:float, t:float, x_uphill: float = 40.0, x_downhill: float = -40.0):
        '''
        Calculate the second rule of space comfort driving on different road grade.
        s_c = c * x ^ 3 + t
    
        Args:
            road_grade: float representing rise over run(vertical change divided by horizontal distance) where positive representing uphill and negative representing downhill.
            y: float representing seconds of the space comfortable driving from front car.
            t: float representing the seconds of the space comfortable driving whend driving on non-sloped road.
            x_uphill: float 40.0% representing the uphill road grade of 40%.
            x_downhill: float -40.0 representing the downhill road grade of 40%.
    
        Returns:
            sec_rule_comfort: second rule of space that the car is comfortable driving.
        '''

        x = x_uphill if road_grade > 0 else x_downhill
        y = y_uphill if road_grade > 0 else y_downhill
        c = self.compute_cubic_constant_final(x,y,t)
  
        # Apply cubic function formula: s_c = c * road_grade^3 + t
        sec_rule_comfort = c * (road_grade ** 3) + t
        return sec_rule_comfort

Car.compute_second_rule_comfort_cubic_final = compute_second_rule_comfort_cubic_final

In [287]:
def plot_sc(self, road_grades,s_c_values1, s_c_values2, s_c_values3):
    # Plot both cars
    plt.figure(figsize=(6, 4))
    # Plot both cars and store the Line2D objects
    line1, = plt.plot(road_grades, s_c_values1, marker='o',linestyle='-',label="cautious driver")
    line2, = plt.plot(road_grades, s_c_values2, marker='s', linestyle='--',label="risker driver")
    line3, = plt.plot(road_grades, s_c_values3, marker='s', linestyle='-',label="aggressive driver")

    # Extract colors assigned to the lines
    color1 = line1.get_color()
    color2 = line2.get_color()
    color3 = line3.get_color()

    # Labels and title
    plt.xlabel("Road Grade (%)")
    plt.ylabel("Second Rule of Comfort(seconds)")
    plt.axhline(y=3.0, color='red', linestyle='--', label="SC(road_grade = 0)")
    plt.axvline(x=0, color='gray', linestyle=':')
    plt.legend()
    #plt.grid(True)
    
    # Show plot
    plt.show()

Car.plot_sc = plot_sc
    

SECOND RULES:
- sec_rule: seconds representing the time for a Car to reach its neighbor.
- sec_rule_comfort: seconds representing the second rule of space for which car is comfortable driving when acceleartion = 0.
- sec_rule_min: seconds representing the second rule of space associated with car need to decelerate by at least a_min to ensure maintaining a distance of at least d_min from front car.
- sec_rule_emer: seconds representing the second rule of space associated with car need to decelerate by a_emer to ensure maintain a distance of d_min from front car.


In [288]:
def convert_time_from_hours_to_seconds(self, second_rule_in_hours: float):
        """
        Convert second rule from hours to seconds.
        
        Args:
            second_rule_in_hours: float representing the second rule in hours
        
        Returns:
            float representing time in seconds
        """
        
        return second_rule_in_hours*3600

Car.convert_time_from_hours_to_seconds = convert_time_from_hours_to_seconds

In [289]:
def convert_acceleration_from_m_s2_to_km_h2(self, acc_in_m_s2: float):
        """
        Convert second rule from hours to seconds.
        
        Args:
            acc_in_m_s2: float representing the acceleration in m/s2
        
        Returns:
            float representing the acceleration in km/h2
        """
        
        return acc_in_m_s2*12960

Car.convert_acceleration_from_m_s2_to_km_h2 = convert_acceleration_from_m_s2_to_km_h2

In [290]:
def compute_sec_rule(self, neighbor_position:float, low_speed: float = 0.1):
    '''
    Compute the time in seconds needed for a car to reach its front car.

    We compute the second rule s for car i as the ratio of the distance between car i and its neighbor and the speed of 
    the Car. We convert s to seconds as the acceleration function a(s) requires s to be in seconds.
        
    Args:
        neighbor_position: float representing the position of the front car.
        low_speed: float = 0.1 m/h representing the lower speed of the current car.
    Returns:
        float in seconds representing the amount of time in seconds needed for a car to reach its neighbor.
    '''

    if self.velocity == 0:
        return self.convert_time_from_hours_to_seconds((neighbor_position - self.position) / low_speed)
    else:
        return self.convert_time_from_hours_to_seconds((neighbor_position - self.position) / self.velocity)

#Extend the class to include the method in the Car class
Car.compute_sec_rule = compute_sec_rule

In [291]:
def compute_sec_rule_min(self, neighbor_velocity: float, neighbor_acceleration: float, low_speed: float = 0.1):
        """
        Compute second rule associated with car i needing to decelerate by at least
        a_min to ensure maintaining at least a distance of d_min from car i-1.
        
        Args:
            neighbor_velocity: float representing the velocity of the front car
            neighbor_acceleration: float representing the acceleration of the front car
        
        Returns:
            float representing the second rule associated with car i needing to 
            decelerate by at least a_min to ensure maintaining at least a distance 
            of d_min from car i-1
        """
        
        if self.velocity == 0:
            return self.convert_time_from_hours_to_seconds(self.convert_acceleration_from_m_s2_to_km_h2(self.a_min)* (self.delta_t**2)/2 
        - neighbor_velocity*self.delta_t
        - self.convert_acceleration_from_m_s2_to_km_h2(neighbor_acceleration)* (self.delta_t**2)/2 
        + low_speed*self.delta_t + self.d_min)/low_speed
        else:
            return self.convert_time_from_hours_to_seconds(self.convert_acceleration_from_m_s2_to_km_h2(self.a_min)* (self.delta_t**2)/2 
        - neighbor_velocity*self.delta_t 
        - self.convert_acceleration_from_m_s2_to_km_h2(neighbor_acceleration)* (self.delta_t**2)/2 
        + self.velocity*self.delta_t + self.d_min)/self.velocity

#Extend the class to include the method in the Car class
Car.compute_sec_rule_min = compute_sec_rule_min

In [292]:
def compute_sec_rule_emer(self, neighbor_velocity: float, neighbor_acceleration: float, low_speed = 0.1):
        """
        Compute second rule associated with car i needing to decelerate by at least
        -a_emer to ensure maintaining at least a distance of d_min from car i-1.
        
        Args:
            neighbor_velocity: float representing the velocity of the front car
            neighbor_acceleration: float representing the acceleration of the front car
        
        Returns:
            float representing the second rule associated with the case where the maximum 
                acceleration/deceleration possible to keep the minimal distance between consecutive 
                cars is the maximum deceleration possible
        """
        if self.velocity == 0:
            return self.convert_time_from_hours_to_seconds(self.convert_acceleration_from_m_s2_to_km_h2(-self.a_emer)* (self.delta_t**2)/2 
        - neighbor_velocity*self.delta_t 
        - self.convert_acceleration_from_m_s2_to_km_h2(neighbor_acceleration)* (self.delta_t**2)/2 
        + low_speed*self.delta_t + self.d_min)/low_speed
        else:
            return self.convert_time_from_hours_to_seconds(self.convert_acceleration_from_m_s2_to_km_h2(-self.a_emer)* (self.dt**2)/2 
        - neighbor_velocity*self.delta_t
        - self.convert_acceleration_from_m_s2_to_km_h2(neighbor_acceleration)* (self.delta_t**2)/2 
        + self.velocity*self.delta_t + self.d_min)/self.velocity

#Extend the class to include the method in the Car class
Car.compute_sec_rule_emer = compute_sec_rule_emer

ACCELERATION MODELING: 
- accel_linear: sec_rule_emergency <= sec_rule < sec_rule_min. acceleration of a Car for second rules smaller than sec_rule_min, that is cases where a driver is breaking uncomfortably.In such cases, a person would always try to minimize its breaking as breaking more would be even more uncomfortable. We use a linear function between the points (sec_rule_min, a_min) and (sec_rule_emergency, -a_emer) as the max acceleration vs second rule is a linear function
- accel_logistic: sec_rule > sec_rule_min. acceleration of a Car for second rules larger than sec_rule_min following the logistic function that models how a Car would accelerate depending on how fast it is driving, how far the Car in front is and how strongly the Car in front is decelerating/accelerating.

In [293]:
def compute_accel_linear(self, neighbor_position: float, neighbor_velocity: float, neighbor_acceleration: float):
        """
        Compute acceleration for second rules smaller than
            sec_rule_min according to the linear model.
            
        Args:
            neighbor_position: float representing the position of the front car
            neighbor_velocity: float representing the velocity of the front car
            neighbor_acceleration: float representing the acceleration of the front car
        
        Returns:
            float representing the acceleration of a Car for second rules smaller than 
                sec_rule_min, that is cases where a driver is breaking uncomfortably.
                In such cases, a person would always try to minimize its breaking as breaking
                more would be even more uncomfortable. We use a linear function between the 
                points (sec_rule_min, a_min) and (sec_rule_emergency, -a_emer) as the
                max acceleration vs second rule is a linear function
        """
        
        sec_rule_min = self.compute_sec_rule_min(neighbor_velocity, neighbor_acceleration)
        sec_rule_emer = self.compute_sec_rule_emer(neighbor_velocity, neighbor_acceleration)
                
        slope = (self.a_min + self.a_emer)/(sec_rule_min - sec_rule_emer)
        intercept = (-self.a_min * sec_rule_emer - sec_rule_min * self.a_emer)/(sec_rule_min - sec_rule_emer)
    
        return slope * self.compute_sec_rule(neighbor_position) + intercept


#Extend the class to include the method in the Car class
Car.compute_accel_linear = compute_accel_linear

In [294]:
def compute_distance_to_front(self, neighbor_position: float):
        """
        Compute distance between the Car instance and its neighbor.
        
        Args:
            neighbor_position: float representing the position of the front car
        
        Returns:
            float representing distance between the Car instance and its neighbor
        """

        return neighbor_position - self.position

#Extend the class to include the method in the Car class
Car.compute_distance_to_front = compute_distance_to_front

In [295]:
def compute_acc_look_ahead(self, neighbor_position: float, neighbor_velocity: float, neighbor_acceleration: float, K: float = 1.05):
        """
        Compute the acceleration for newly inserted cars using a look ahead strategy 
                
        Args:
            neighbor_position: float representing the position of the front car
            neighbor_velocity: float representing the velocity of the front car
            neighbor_acceleration: float representing the acceleration of the front car
            K: float representing the decay in the second rule to ensure a gradual decrease
                in speed
        
        Returns:
            float representing the acceleration needed to ensure that s_i^{t+1} = K*s_i^t
             ensuring that that the car slows down gradually
        """

        accel_look_ahead_num = self.velocity*((1-K)*self.compute_distance_to_front(neighbor_position) + 
                                              (neighbor_velocity - self.velocity)*self.delta_t + 
                                              neighbor_acceleration*(self.delta_t**2)/2)
    
        accel_look_ahead_denom = K*self.delta_t*self.compute_distance_to_front(neighbor_position) + self.velocity*(self.delta_t**2)/2
                       
        return (accel_look_ahead_num/accel_look_ahead_denom)/12960

#Extend the class to include the method in the Car class
Car.compute_acc_look_ahead = compute_acc_look_ahead

In [296]:
def compute_accel_logistic(self, neighbor_position: float, neighbor_velocity: float, neighbor_acceleration: float, road_grade: float, sc_uphill: float, sc_downhill: float, sc_flat: float, eps: float = 0.01):
        """
        Compute acceleration for second rules larger than sec_rule_min according to the logistic model.
            
        Args:
            neighbor_position: float representing the position of the front car
            neighbor_velocity: float representing the velocity of the front car
            neighbor_acceleration: float representing the acceleration of the front car
            
        Parameters:
            low_asymp: float representing the lower asymptote.
            up_asymp: float representing the upper asymptote.
            Q: float computed to make sure that at the critical second rule, the 
                function value is 0.
            growth_rate: float representing the slope of the "linear" part in the logistic function
        
        Returns:
            float representing the acceleration of a Car for second rules larger than 
                sec_rule_min following the logistic function that models how a Car 
                would accelerate depending on how fast it is driving, how far the Car 
                in front is and how strongly the Car in front is decelerating/accelerating.
        """
        
        low_asymp = self.a_min - eps
        up_asymp = self.a_max
        Q = (up_asymp - low_asymp)*(1/eps)
        sec_rule_min = self.compute_sec_rule_min(neighbor_velocity, neighbor_acceleration)

        #Update the sec_rule_comfort from a floating parameter in Car class to a function.
        sec_rule_comfort = self.compute_second_rule_comfort_cubic_final(road_grade, sc_uphill, sc_downhill, sc_flat)
    
        if (self.velocity > neighbor_velocity) and (self.velocity != 0) and (self.compute_sec_rule(neighbor_position) > sec_rule_comfort):
            return self.compute_acc_look_ahead(neighbor_position, neighbor_velocity, neighbor_acceleration)
        '''
        else:
            sec_rule_crit = self.sec_rule_crit_comfort
        '''
    
        growth_rate = np.log(self.a_max/(-Q*low_asymp))/(sec_rule_min - sec_rule_comfort)

        return low_asymp + (up_asymp-low_asymp)/(1+ Q*np.exp(-growth_rate*(self.compute_sec_rule(neighbor_position)-sec_rule_min)))

#Extend the class to include the method in the Car class
Car.compute_accel_logistic = compute_accel_logistic

In [297]:
def compute_acceleration(self, neighbor_position: float, neighbor_velocity: float, neighbor_acceleration: float, road_grade: float, sc_uphill: float, sc_downhill: float, sc_flat: float,):
        """
        Compute the acceleration of a Car using either the linear 
            or logistic function.

        Args:
            neighbor_position: float representing the position of the front car
            neighbor_velocity: float representing the velocity of the front car
            neighbor_acceleration: float representing the acceleration of the front car
            
        If car i has a neighbor, we use our model a(s) to find the acceleration.
        Otherwise, car i keeps on accelerating until reaching the max speed and
        having an acceleration of 0. We check that both cars i and i-1 are driving
        at max speed before setting a_i^t = 0. The last if statement is included
        to take care of floating point errors.
        
        Returns:
            float representing the acceleration based on either the logistic or
                linear function, depending on the sec_rule
        """
        
        if self.neighbor is None:
            if self.velocity < self.max_speed:
                acceleration = self.a_max
            else:
                acceleration = 0
        else:
            if self.velocity == self.max_speed and neighbor_velocity == self.max_speed:
                acceleration = 0
            else:
                sec_rule = self.compute_sec_rule(neighbor_position)
                sec_rule_min = self.compute_sec_rule_min(neighbor_velocity, neighbor_acceleration)
                sec_rule_emer = self.compute_sec_rule_emer(neighbor_velocity, neighbor_acceleration)
                
                if (sec_rule <= sec_rule_min and sec_rule >= sec_rule_emer):
                    acceleration = self.compute_accel_linear(neighbor_position, neighbor_velocity, neighbor_acceleration)

                elif sec_rule < sec_rule_emer:
                    acceleration = -self.a_emer
                    print("Cars had to be within d_min as the max deceleration was not enough to keep a distance of at least d_min")
                else:
                    acceleration = self.compute_accel_logistic(neighbor_position, neighbor_velocity, neighbor_acceleration, road_grade,sc_uphill, sc_downhill, sc_flat)
                    
                    
        if np.abs(acceleration) < 10**(-5):
            acceleration = 0
            
        return acceleration

#Extend the class to include the method in the Car class
Car.compute_acceleration = compute_acceleration

Note: The following are the different version of the acceleration model received on 4/1.

In [298]:
def compute_sec_rule_minus_2(self):
            """
            Compute second rule associated with the case where the maximum acceleration possible to
                keep the minimal distance between consecutive cars is -2.

            Returns:
                float representing the second rule associated with the case where the maximum 
                    acceleration possible to keep the minimal distance between consecutive 
                    cars is -2.
            """

            return self.convert_time_from_hours_to_seconds(self.convert_acceleration_from_m_s2_to_km_h2(self.a_min)* (self.delta_t**2)/2 
            - self.velocity1*self.delta_t
            - self.convert_acceleration_from_m_s2_to_km_h2(self.acc1)* (self.delta_t**2)/2 
            + self.velocity2*self.delta_t + self.d_min)/self.velocity2

Car.compute_sec_rule_minus_2 = compute_sec_rule_minus_2

In [299]:
def compute_sec_rule_emergency_update(self):
        """
        Compute second rule associated with the case where the maximum acceleration possible to
            keep the minimal distance between consecutive cars is the deceleration limit.
        
        Returns:
            float representing the second rule associated with the case where the maximum 
                acceleration/deceleration possible to keep the minimal distance between consecutive 
                cars is the maximum deceleration possible.
        """
        return self.convert_time_from_hours_to_seconds(self.convert_acceleration_from_m_s2_to_km_h2(self.a_emer)* (self.delta_t**2)/2 
        - self.velocity1*self.delta_t
        - self.convert_acceleration_from_m_s2_to_km_h2(self.acc1)* (self.delta_t**2)/2 
        + self.velocity2*self.delta_t + self.d_min)/self.velocity2

Car.compute_sec_rule_emergency_update = compute_sec_rule_emergency_update

In [300]:
def compute_logistic_function_update(self, sec_rule, road_grade: float, sc_uphill: float, sc_downhill: float, sc_flat: float, eps: float = 0.01):
        """
        Compute logistic function modeling the acceleration for second rules larger than
            sec_rule_minus_2.
            
        Parameters:
            A: float representing the lower asymptote.
            K: float representing the upper asymptote.
            C: float related to to the upper and lower asymptote
            Q: float computed to make sure that at the critical second rule, the 
                function value is 0.
            B: float representing the slope of the "linear" part in the logistic function
        
        Returns:
            float representing the acceleration of a Car for second rules larger than 
                sec_rule_minus_2 following the logistic function that models how a Car 
                would accelerate depending on how fast it is driving, how far the Car 
                in front is and how strongly the Car in front is decelerating/accelerating.
        """
        
        sec_rule_minus_2 = self.compute_sec_rule_minus_2()
                    
        sec_rule_crit = self.compute_second_rule_comfort_cubic_final(road_grade, sc_uphill, sc_downhill, sc_flat)
        

        A = self.a_min - eps
        K = self.a_max
        C = 1
        Q = (K - A)*(1/eps)
        B = np.log(self.a_max/(-Q*A))/(sec_rule_minus_2 - sec_rule_crit)

        return A + (K-A)/(C+ Q*np.exp(-B*(sec_rule-sec_rule_minus_2)))

Car.compute_logistic_function_update = compute_logistic_function_update

In [301]:
def compute_linear_function_update(self, sec_rule):
        """
        Compute linear function modeling the acceleration for second rules smaller than
            sec_rule_minus_2.
        
        Returns:
            float representing the acceleration of a Car for second rules smaller than 
                sec_rule_minus_2, that is cases where a driver is breaking uncomfortably.
                In such cases, a person would always try to maximize its breaking as breaking
                more would be even more uncomfortable. We use a linear function between the 
                points (sec_rule_minus_2, -2) and (sec_rule_emergency, -acc_max) as the
                max acceleration vs second rule is a linear function.
        """
        
        sec_rule_minus_2 = self.compute_sec_rule_minus_2()
        sec_rule_emergency = self.compute_sec_rule_emergency_update()
        
        slope = (self.a_min - self.a_emer)/(sec_rule_minus_2 - sec_rule_emergency)
        intercept = (-self.a_min*sec_rule_emergency + sec_rule_minus_2*self.a_emer)/(sec_rule_minus_2 - sec_rule_emergency)
    
        return slope*sec_rule + intercept

Car.compute_linear_function_udpate = compute_linear_function_update

In [302]:
def compute_acceleration_update(self,road_grade: float, sc_uphill: float, sc_downhill: float, sc_flat: float):
        """
        Compute the acceleration of a Car based on function composed of the linear 
            and logistic function.
        
        If the Car instance has a neighbor, we use the function to find the acceleration.
        Otherwise, we just keep on accelerating. Once a Car reaches the max speed and its
        neighbor as well, the Car's acceleration would be 0.
        
        Returns:
            float representing the acceleration based on either the logistic function or
                linear function, depending on the sec_rule.
        """
        
        accelerations = np.zeros_like(self.sec_rule)
        
        for i in range(len(accelerations)):        
            if self.sec_rule[i] <= self.compute_sec_rule_minus_2():
                accelerations[i] = self.compute_linear_function_update(self.sec_rule[i])
            else:
                accelerations[i] = self.compute_logistic_function_update(self.sec_rule[i],road_grade, sc_uphill, sc_downhill, sc_flat)
            #print(f"sec_rule: {self.sec_rule[i]}")
            #print(f"sec_rule_minus_2: {self.compute_sec_rule_minus_2()}")
                
        return accelerations

Car.compute_acceleration_update = compute_acceleration_update

In [303]:
def plot_acceleration_sec_rule(self, sec_rule_cleaned: list[float], acc_cleaned: list[float]):
    plt.plot(sec_rule_cleaned3_downhill, acc_cleaned3_downhill, label = "cautious driver")
    plt.plot(sec_rule_cleaned4_downhill, acc_cleaned4_downhill, label = "risker driver")
    plt.plot(sec_rule_cleaned5_downhill, acc_cleaned5_downhill, label = "aggressive driver")
    plt.xlabel("Second rule " r"$s$", fontsize=17)
    plt.ylabel("Acceleration " r"$a(s)$", fontsize=17)
    plt.axhline(y=0.0, color='gray', linestyle='--', label="SC(road_grade = 0)")
    plt.xticks(np.arange(0, 6, 1))
    plt.legend(fontsize=14)
    plt.xticks(fontsize=17)
    plt.yticks(fontsize=17)
    plt.tight_layout()

Car.plot_acceleration_sec_rule = plot_acceleration_sec_rule

In [304]:
def plot_acceleration_by_driver(self,s_vecs: list[list[float]], a_vecs: list[list[float]], labels: list[str], title: str):
    """
    Plots acceleration vs second rule for multiple drivers.

    Args:
        s_vecs: List of second rule lists (x-axis values).
        a_vecs: List of acceleration lists (y-axis values).
        labels: List of labels for each driver.
        title: Title for the plot.
    """
    plt.figure(figsize=(6, 4))

    for sec_rules, accs, label in zip(s_vecs, a_vecs, labels):
        plt.plot(sec_rules, accs, label=label)

    plt.xlabel("Second Rule " r"$(s)$", fontsize=12)
    plt.ylabel("Acceleration " r"$(km/h/s)$", fontsize=12)
    plt.axhline(y=0.0, color='gray', linestyle='--', label="Zero acceleration")
    plt.xticks(np.arange(0, 6, 1), fontsize=12)
    plt.yticks(fontsize=12)
    plt.title(title, fontsize=14)
    plt.legend(fontsize=10)
    plt.tight_layout()
    plt.show()

Car.plot_acceleration_by_driver = plot_acceleration_by_driver

In [305]:
def clean_acceleration_data(acc, sec_rules, threshold=-5.0):
    acc_cleaned = []
    sec_cleaned = []
    for i in range(len(acc)):
        if acc[i] >= threshold:
            acc_cleaned.append(acc[i])
            sec_cleaned.append(sec_rules[i])
    return sec_cleaned, acc_cleaned

Car.clean_acceleration_data = clean_acceleration_data

NOT USED - old version of sec rule of comfort

In [306]:
'''
def compute_second_rule_comfort_cubic(self,road_grade:float):
    Calculate the second rule of space comfort driving on different road grade.
    s_c = c * x ^ 3 + t

    Args:
        road_grade: float representing rise over run(vertical change divided by horizontal distance) where positive representing uphill and negative representing downhill.
        t: float = 3.0 second rule of space comfortable driving when driving on flat.

    Returns:
        sec_rule_comfort: second rule of space that the car is comfortable driving.


    #Case 1 & 2: road grade = 40 uphill, aggressive driver would takes 1 second and non-aggressive drivers would take 2 seconds
    if road_grade >0:
        x = 40
        y = 2.0 if not self.aggressive_driver else 1.0
        
    #Case 3 & 4: road grade = 40 downhill, aggressive driver would take 3 seconds and non-aggressive drivers would take 3 seconds.
    else:
        x = -40
        y = 5.0 if not self.aggressive_driver else 3.0   
        
    #Case 5 & 6: road grade = 0, aggressive driver would take 2 seconds and non-aggressive drivers would take 2 seconds.
    t = 3.0 if not self.aggressive_driver else 2.0
    
    c = self.compute_cubic_constant_final(x, y, t)

    # Apply cubic function formula: s_c = c * road_grade^3 + t
    sec_rule_comfort = c * (road_grade ** 3) + t
    return sec_rule_comfort

Car.compute_second_rule_comfort_cubic = compute_second_rule_comfort_cubic
'''

'\ndef compute_second_rule_comfort_cubic(self,road_grade:float):\n    Calculate the second rule of space comfort driving on different road grade.\n    s_c = c * x ^ 3 + t\n\n    Args:\n        road_grade: float representing rise over run(vertical change divided by horizontal distance) where positive representing uphill and negative representing downhill.\n        t: float = 3.0 second rule of space comfortable driving when driving on flat.\n\n    Returns:\n        sec_rule_comfort: second rule of space that the car is comfortable driving.\n\n\n    #Case 1 & 2: road grade = 40 uphill, aggressive driver would takes 1 second and non-aggressive drivers would take 2 seconds\n    if road_grade >0:\n        x = 40\n        y = 2.0 if not self.aggressive_driver else 1.0\n        \n    #Case 3 & 4: road grade = 40 downhill, aggressive driver would take 3 seconds and non-aggressive drivers would take 3 seconds.\n    else:\n        x = -40\n        y = 5.0 if not self.aggressive_driver else 3.0  

NOT USED - SECOND RULE OF COMFORT USING LINEAR APPROACH
- This approach was not used as the cubic function better fits the following distance on driving uphill, downhilletc.

In [307]:
'''
def compute_second_rule_of_comfort_linear(road_grade: float):

    compute the second rule of space for which the car is comfortable driving.
    TO DISUCSS: UNABLE TO FIND THE REFERENCE TO SUPPORT THE CHANGE OF SLOPE AND SC.
    
    Args:
        road_grade: floating represent rise over run(vertical change divided by horizontal distance) where positive representing uphill and negative representing downhill.

    Returns:
        s_c: second rule of space that the car is comfortable driving.

    #Base: 3-second rule on flat ground
    s_c_flat = 3.0

    #Case1: Increase second rule of space by 0.5 seconds per 5% grade drop on downhill
    if road_grade < -5:
        s_c = s_c_flat + 0.5 * ((abs(road_grade) - 5) /5)
    #Case2: Decrease second rule of space by 0.1 seconds per every 5% increase on uphill.
    elif road_grade > 5:
        s_c = s_c_flat - 0.1 * (((road_grade) - 5) /5)
    #Case3: maintain 3 seconds rule of space between -5% to 5% grade.
    else:
        s_c = s_c_flat

    #ensure the s_c does not go below the second of rule of emergency (1.5 s).
    #s_c = max(s_c, 1.5)

    return s_c


# Initialize previous value for computing change
previous_s_c = None

# Print table header
print(f"{'road_grade':>12} | {'second_rule_of_space':>22} | {'change':>10}")
print("-" * 50)

# Iterate correctly through road grades from -40% to 40%
for rg in range(-40, 41):  
    s_c = compute_second_rule_of_comfort_linear(rg)

    # Compute the change in `s_c` from the previous row
    change_in_s_c = s_c - previous_s_c if previous_s_c is not None else None

    # Print the table
    if change_in_s_c is None:
        print(f"{rg:>12} | {s_c:>22.10f} | {'-':>10}")  # First row has no previous value
    else:
        print(f"{rg:>12} | {s_c:>22.10f} | {change_in_s_c:>10.10f}")
    
    # Update previous_s_c for the next iteration
    previous_s_c = s_c
'''

'\ndef compute_second_rule_of_comfort_linear(road_grade: float):\n\n    compute the second rule of space for which the car is comfortable driving.\n    TO DISUCSS: UNABLE TO FIND THE REFERENCE TO SUPPORT THE CHANGE OF SLOPE AND SC.\n    \n    Args:\n        road_grade: floating represent rise over run(vertical change divided by horizontal distance) where positive representing uphill and negative representing downhill.\n\n    Returns:\n        s_c: second rule of space that the car is comfortable driving.\n\n    #Base: 3-second rule on flat ground\n    s_c_flat = 3.0\n\n    #Case1: Increase second rule of space by 0.5 seconds per 5% grade drop on downhill\n    if road_grade < -5:\n        s_c = s_c_flat + 0.5 * ((abs(road_grade) - 5) /5)\n    #Case2: Decrease second rule of space by 0.1 seconds per every 5% increase on uphill.\n    elif road_grade > 5:\n        s_c = s_c_flat - 0.1 * (((road_grade) - 5) /5)\n    #Case3: maintain 3 seconds rule of space between -5% to 5% grade.\n    else

In [308]:
'''
def plot_second_rule_linear():
    # Generate road grade values from -40 to 40
    road_grades = np.arange(-40, 41, 1)  # Range from -40 to 40 in steps of 1
    s_c_values = [compute_second_rule_of_comfort_linear(rg) for rg in road_grades]

    # Create the plot
    plt.figure(figsize=(8, 5))
    plt.plot(road_grades, s_c_values, marker='o', linestyle='-', color='b', label="s_c vs. road_grade")

    #set y-axis limits explicitly
    plt.ylim(2,7)
    
    # Labels and title
    plt.xlabel("Road Grade (%)")
    plt.ylabel("Second Rule of Space (s_c)")
    plt.title("Effect of Road Grade on Second Rule of Space")
    plt.axhline(y=3.0, color='r', linestyle='--', label="Flat Ground s_c (3.0)")  # Reference line for flat ground
    plt.axvline(x=0, color='gray', linestyle=':')  # Vertical line at road grade 0
    plt.legend()
    plt.grid(True)
    
    # Show the plot
    plt.show()

#call the function
plot_second_rule_linear()
'''

'\ndef plot_second_rule_linear():\n    # Generate road grade values from -40 to 40\n    road_grades = np.arange(-40, 41, 1)  # Range from -40 to 40 in steps of 1\n    s_c_values = [compute_second_rule_of_comfort_linear(rg) for rg in road_grades]\n\n    # Create the plot\n    plt.figure(figsize=(8, 5))\n    plt.plot(road_grades, s_c_values, marker=\'o\', linestyle=\'-\', color=\'b\', label="s_c vs. road_grade")\n\n    #set y-axis limits explicitly\n    plt.ylim(2,7)\n    \n    # Labels and title\n    plt.xlabel("Road Grade (%)")\n    plt.ylabel("Second Rule of Space (s_c)")\n    plt.title("Effect of Road Grade on Second Rule of Space")\n    plt.axhline(y=3.0, color=\'r\', linestyle=\'--\', label="Flat Ground s_c (3.0)")  # Reference line for flat ground\n    plt.axvline(x=0, color=\'gray\', linestyle=\':\')  # Vertical line at road grade 0\n    plt.legend()\n    plt.grid(True)\n    \n    # Show the plot\n    plt.show()\n\n#call the function\nplot_second_rule_linear()\n'

In [309]:
'''
def convert_road_grade_to_angle(road_grade: float):

    Convert road grade (%) to incline angle in degrees.
    Formula: θ = arctan(grade / 100)

    Args:
        road_grade: floating represent rise over run(vertical change divided by horizontal distance) where positive representing uphill and negative representing downhill.

    Return:
        float: representing the incline angle in degrees.

    return math.degrees(math.atan(road_grade /100))

#test with examples
road_grades = [1.0, 5.0, 10.0, 20.0, 30.0, 40.0]

for rg in road_grades:
    angle = convert_road_grade_to_angle(rg)
    print(f"road_grade {rg}% convert to angle:{angle:.10f}")
'''  

'\ndef convert_road_grade_to_angle(road_grade: float):\n\n    Convert road grade (%) to incline angle in degrees.\n    Formula: θ = arctan(grade / 100)\n\n    Args:\n        road_grade: floating represent rise over run(vertical change divided by horizontal distance) where positive representing uphill and negative representing downhill.\n\n    Return:\n        float: representing the incline angle in degrees.\n\n    return math.degrees(math.atan(road_grade /100))\n\n#test with examples\nroad_grades = [1.0, 5.0, 10.0, 20.0, 30.0, 40.0]\n\nfor rg in road_grades:\n    angle = convert_road_grade_to_angle(rg)\n    print(f"road_grade {rg}% convert to angle:{angle:.10f}")\n'

In [310]:
'''
def convert_road_grade_to_radians(road_grade: float):
    return math.atan(road_grade / 100)

#test with examples
road_grades = [1.0, 5.0, 10.0, 20.0, 30.0, 40.0]

for rg in road_grades:
    angle = convert_road_grade_to_radians(rg)
    print(f"road_grade {rg}% convert to radians:{angle:.10f}")
'''

'\ndef convert_road_grade_to_radians(road_grade: float):\n    return math.atan(road_grade / 100)\n\n#test with examples\nroad_grades = [1.0, 5.0, 10.0, 20.0, 30.0, 40.0]\n\nfor rg in road_grades:\n    angle = convert_road_grade_to_radians(rg)\n    print(f"road_grade {rg}% convert to radians:{angle:.10f}")\n'


NOT USED - CALCULATION OF ACCELERATION FROM EXTRA FORCE ON UPHILL/DOWNWHILL
- Approach 1: Apply road angle to gravity and normal force to calculate the acceleration/deceleration from engine to keep on same speed.
Note: This approach was verified to be not an appropriate apporach for our study 

In [311]:
'''
def compute_acceleration_extra_force(road_grade: float, mu_k: float = 0.75, gravity: float = 9.81):

    Compute the required acceleration/deceleartion on uphill/dowhill to maintain intial speed due to friction and normal force.

    Formula: 
        uphill: a = -g * (μk * cosθ + sinθ)
        downhill: a = g * (sinθ - μk * cosθ )

    Args:
        mu_k: float (0.01 - 0.15) representing rolling resistance of ordinary car tire on concrete
        road_grade: floating represent road grade.
        gravity: float representing gravitational acceleration 9.81 m/s².

    Return:
        float: m/s² representing the acceleration from extra force to maintain constant speed when driving uphill/downhill.

    #convert the road grade % to angle
    theta = convert_road_grade_to_radians(road_grade)

    #calculate the acceleration from extra force considering friction and normal force in uphills/downhills
    if road_grade >= 0:
        acc_extra_force =  -gravity * (mu_k * math.cos(theta) + math.sin(theta))
    else:
        acc_extra_force = gravity * ( math.sin(theta)- mu_k * math.cos(theta) )

    return acc_extra_force
'''

'\ndef compute_acceleration_extra_force(road_grade: float, mu_k: float = 0.75, gravity: float = 9.81):\n\n    Compute the required acceleration/deceleartion on uphill/dowhill to maintain intial speed due to friction and normal force.\n\n    Formula: \n        uphill: a = -g * (μk * cosθ + sinθ)\n        downhill: a = g * (sinθ - μk * cosθ )\n\n    Args:\n        mu_k: float (0.01 - 0.15) representing rolling resistance of ordinary car tire on concrete\n        road_grade: floating represent road grade.\n        gravity: float representing gravitational acceleration 9.81 m/s².\n\n    Return:\n        float: m/s² representing the acceleration from extra force to maintain constant speed when driving uphill/downhill.\n\n    #convert the road grade % to angle\n    theta = convert_road_grade_to_radians(road_grade)\n\n    #calculate the acceleration from extra force considering friction and normal force in uphills/downhills\n    if road_grade >= 0:\n        acc_extra_force =  -gravity * (mu_k

In [312]:
'''
#Test the compute_acceleration_extra_force function
road_grade_vals = np.linspace(-5, 5, 9)
acc_extra_force_all = []
for road_grade in road_grade_vals:
    acc_extra_force = compute_acceleration_extra_force(road_grade = road_grade)
    acc_extra_force_all.append(acc_extra_force)
    print(f"road_grade {road_grade}")
    print(f"acc_extra_force {acc_extra_force}")
    
plt.scatter(road_grade_vals, acc_extra_force_all) 

acc_extra_force1 = compute_acceleration_extra_force(road_grade = 20.0)
acc_extra_force1 = compute_acceleration_extra_force(road_grade = 3.0)
acc_extra_force2 = compute_acceleration_extra_force(road_grade = -10.0)
acc_extra_force1 = compute_acceleration_extra_force(road_grade = -3.0)
print(f"Required acceleration is : {acc_extra_force1:.5f} m/s²")
print(f"Required acceleration is : {acc_extra_force2:.5f} m/s²")

'''

'\n#Test the compute_acceleration_extra_force function\nroad_grade_vals = np.linspace(-5, 5, 9)\nacc_extra_force_all = []\nfor road_grade in road_grade_vals:\n    acc_extra_force = compute_acceleration_extra_force(road_grade = road_grade)\n    acc_extra_force_all.append(acc_extra_force)\n    print(f"road_grade {road_grade}")\n    print(f"acc_extra_force {acc_extra_force}")\n    \nplt.scatter(road_grade_vals, acc_extra_force_all) \n\nacc_extra_force1 = compute_acceleration_extra_force(road_grade = 20.0)\nacc_extra_force1 = compute_acceleration_extra_force(road_grade = 3.0)\nacc_extra_force2 = compute_acceleration_extra_force(road_grade = -10.0)\nacc_extra_force1 = compute_acceleration_extra_force(road_grade = -3.0)\nprint(f"Required acceleration is : {acc_extra_force1:.5f} m/s²")\nprint(f"Required acceleration is : {acc_extra_force2:.5f} m/s²")\n\n'

CALCULATION OF ACCELERATION FROM EXTRA FORCE ON UPHILL/DOWNWHILL
- Approach 2: acceleration dropping by -0.05 to -0.075 mph/s (or -0.0224 m/s² to -0.0335m/s²) for every 1% increase of grade.
source: https://www.sciencedirect.com/science/article/abs/pii/S1361920918307260

In [313]:
'''
def compute_acceleration_extra_force_approach2(delta_grade: float, theta_0_v: float = 0.0, theta_1_v: float = - 0.0224):

    Compute the required acceleration/deceleartion on uphill/dowhill to maintain intial speed due to friction and normal force.
    Approach 2: acceleration dropping by -0.05 to -0.075 mph/s (or -0.0224 m/s² to -0.0335m/s²) for every 1% increase of grade.
    Formula: 
        Accv   = θ_0_v  +  θ_1_v * Grade + εv 

    Args:
        theta_0_v: 0.0 m/s² float representing flat road acceleration
        theta_1_v: -0.0224 m/s² float representing acceleration reduction per 1% increase on grade.
        delta_grade: % change of the road grade. Position representing uphill and negative representing downhill.

    Return:
        float: m/s² representing the acceleration from extra force to maintain constant speed when driving uphill/downhill.

    acc_extra_force = theta_0_v + theta_1_v * delta_grade
    return - acc_extra_force 
''' 

'\ndef compute_acceleration_extra_force_approach2(delta_grade: float, theta_0_v: float = 0.0, theta_1_v: float = - 0.0224):\n\n    Compute the required acceleration/deceleartion on uphill/dowhill to maintain intial speed due to friction and normal force.\n    Approach 2: acceleration dropping by -0.05 to -0.075 mph/s (or -0.0224 m/s² to -0.0335m/s²) for every 1% increase of grade.\n    Formula: \n        Accv   = θ_0_v  +  θ_1_v * Grade + εv \n\n    Args:\n        theta_0_v: 0.0 m/s² float representing flat road acceleration\n        theta_1_v: -0.0224 m/s² float representing acceleration reduction per 1% increase on grade.\n        delta_grade: % change of the road grade. Position representing uphill and negative representing downhill.\n\n    Return:\n        float: m/s² representing the acceleration from extra force to maintain constant speed when driving uphill/downhill.\n\n    acc_extra_force = theta_0_v + theta_1_v * delta_grade\n    return - acc_extra_force \n'

In [314]:
'''
#Test the compute_acceleration_extra_force_approach2 function
acc_extra_force = compute_acceleration_extra_force_approach2(delta_grade = 20.0)
print(f"Required acceleration is : {acc_extra_force:.5f} m/s²")
'''

'\n#Test the compute_acceleration_extra_force_approach2 function\nacc_extra_force = compute_acceleration_extra_force_approach2(delta_grade = 20.0)\nprint(f"Required acceleration is : {acc_extra_force:.5f} m/s²")\n'